In [81]:
from googleapiclient.discovery import build
import pandas as pd
import json
import time

In [3]:
restaurants_notmerged = pd.read_pickle('./yelp_dataset_processed/yelp_df_notmerged.pkl')
restaurants_old = pd.read_pickle('./yelp_dataset_processed/yelp_df_found.pkl')

In [4]:
restaurants_old.head()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,found
0,PzOqRohWw7F7YEPBz6AubA,"[Food, Bagels, Delis, Restaurants]",Glendale,"6520 W Happy Valley Rd\nSte 101\nGlendale Az, ...",33.712797,-112.200264,Hot Bagels & Deli,[],True,14,3.5,AZ,business,1
1,qarobAbxGSHI7ygf1f7a_Q,"[Sandwiches, Restaurants]",Gilbert,"891 E Baseline Rd\nSuite 102\nGilbert, AZ 85233",33.378839,-111.812007,Jersey Mike's Subs,[],True,10,3.5,AZ,business,1
2,gA5CuBxF-0CnOpGnryWJdQ,"[Mexican, Restaurants]",Phoenix,"519 W Thomas Rd\nPhoenix, AZ 85013",33.480105,-112.081361,La Paloma Mexican Food,[],True,5,4.0,AZ,business,0
3,JxVGJ9Nly2FFIs_WpJvkug,"[Pizza, Restaurants]",Scottsdale,"14418 N Scottsdale Rd\nSuite 181\nScottsdale, ...",33.617459,-111.926272,Sauce,[],True,55,4.0,AZ,business,1
4,Jj7bcQ6NDfKoz4TXwvYfMg,"[Burgers, Restaurants]",Phoenix,"8941 N Black Canyon Hwy\nPhoenix, AZ 85021",33.566989,-112.116243,Fuddruckers,[],True,23,4.0,AZ,business,1


In [5]:
restaurants_notmerged.head()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,found
2,gA5CuBxF-0CnOpGnryWJdQ,"[Mexican, Restaurants]",Phoenix,"519 W Thomas Rd\nPhoenix, AZ 85013",33.480105,-112.081361,La Paloma Mexican Food,[],True,5,4.0,AZ,business,0
7,sygb2W4NRadVXiWhW8bnFQ,"[Mexican, Restaurants]",Gila Bend,"616 E Pima Street\nGila Bend, AZ 85337",32.948295,-112.711322,Sofia's Mexican Food,[],True,10,3.5,AZ,business,0
10,MG1IbQUurcXn8NQgXGM34Q,"[Delis, Restaurants]",Mesa,"4410 E University Dr\nMesa, AZ 85205",33.423805,-111.735789,Yee-haws,[],True,3,3.5,AZ,business,0
11,KqYY2rwD8ymjU9mVJwiuRQ,"[Chinese, Restaurants]",Phoenix,"9020 N 19th Ave\nPhoenix, AZ 85021",33.568297,-112.099840,Panda Gourmet,[],True,3,3.5,AZ,business,0
12,jvvh4Q00Hq2XyIcfmAAT2A,"[Burgers, Restaurants]",Scottsdale,"7014-590 E Camelback Rd\nScottsdale, AZ 85251",33.503323,-111.928457,Five Guys Burger and Fries,[],True,52,3.5,AZ,business,0


In [6]:
restaurants_old_notmerged = restaurants_old[restaurants_old['found'] == 0]

In [7]:
# These two should contain the same data
print(len(restaurants_old_notmerged),len(restaurants_notmerged))

1299 1299


In [10]:
# Read keys
with open('google_credentials.json') as data_file:    
    cred = json.load(data_file)


In [11]:
# Test Google API
my_api_key = cred['API_Key']
my_cse_id = cred['CSE_id']

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res['items']

results = google_search(
    'La Paloma Mexican Food 519 W Thomas Rd Phoenix, AZ 85013', my_api_key, my_cse_id, num=10)
for result in results:
    print(result)

{'kind': 'customsearch#result', 'title': 'La Paloma Mexican Food - CLOSED - 12 Photos & 18 Reviews ...', 'htmlTitle': '<b>La Paloma Mexican Food</b> - CLOSED - 12 Photos &amp; 18 Reviews ...', 'link': 'https://www.yelp.com/biz/la-paloma-mexican-food-phoenix', 'displayLink': 'www.yelp.com', 'snippet': 'Mexican ... 519 W Thomas Rd Phoenix, AZ 85013. Get Directions ... Photo of La \nPaloma Mexican Food - Phoenix, AZ, United States. Andrea,. Chris E. Andrea\xa0...', 'htmlSnippet': 'Mexican ... <b>519 W Thomas Rd Phoenix</b>, <b>AZ 85013</b>. Get Directions ... Photo of <b>La</b> <br>\n<b>Paloma Mexican Food</b> - Phoenix, AZ, United States. Andrea,. Chris E. Andrea&nbsp;...', 'cacheId': 'jauJFZTdrNEJ', 'formattedUrl': 'https://www.yelp.com/biz/la-paloma-mexican-food-phoenix', 'htmlFormattedUrl': 'https://<b>www</b>.yelp.com/biz/<b>la</b>-<b>paloma</b>-<b>mexican</b>-<b>food</b>-<b>phoenix</b>', 'pagemap': {'cse_thumbnail': [{'width': '259', 'height': '194', 'src': 'https://encrypted-tbn0.g

In [17]:
results[0]

{'cacheId': 'jauJFZTdrNEJ',
 'displayLink': 'www.yelp.com',
 'formattedUrl': 'https://www.yelp.com/biz/la-paloma-mexican-food-phoenix',
 'htmlFormattedUrl': 'https://<b>www</b>.yelp.com/biz/<b>la</b>-<b>paloma</b>-<b>mexican</b>-<b>food</b>-<b>phoenix</b>',
 'htmlSnippet': 'Mexican ... <b>519 W Thomas Rd Phoenix</b>, <b>AZ 85013</b>. Get Directions ... Photo of <b>La</b> <br>\n<b>Paloma Mexican Food</b> - Phoenix, AZ, United States. Andrea,. Chris E. Andrea&nbsp;...',
 'htmlTitle': '<b>La Paloma Mexican Food</b> - CLOSED - 12 Photos &amp; 18 Reviews ...',
 'kind': 'customsearch#result',
 'link': 'https://www.yelp.com/biz/la-paloma-mexican-food-phoenix',
 'pagemap': {'aggregaterating': [{'ratingvalue': '3.0', 'reviewcount': '18'},
   {'ratingvalue': '3.0', 'reviewcount': '18'}],
  'breadcrumb': [{'title': 'Restaurants', 'url': 'Restaurants'},
   {'title': 'Mexican', 'url': 'Mexican'}],
  'cse_image': [{'src': 'https://s3-media4.fl.yelpcdn.com/bphoto/qUjY7xiEYKjZG3P8WUezlQ/o.jpg'}],
  'c

In [18]:
dir(results[0])

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [87]:
results[0]['link'].split('/')[3]

'biz'

In [29]:
# prepare input
restaurants_notmerged.loc[2,'name']+' '+restaurants_notmerged.loc[2,'full_address'].replace('\n',' ')

'La Paloma Mexican Food 519 W Thomas Rd Phoenix, AZ 85013'

In [33]:
# apply it on the whole column, no for loops
restaurants_notmerged['name']+' '+restaurants_notmerged['full_address'].str.replace('\n',' ')

2       La Paloma Mexican Food 519 W Thomas Rd Phoenix...
7       Sofia's Mexican Food 616 E Pima Street Gila Be...
10           Yee-haws 4410 E University Dr Mesa, AZ 85205
11        Panda Gourmet 9020 N 19th Ave Phoenix, AZ 85021
12      Five Guys Burger and Fries 7014-590 E Camelbac...
13      Forefathers Gourmet Cheesesteaks & Fries 8707 ...
14             Subway 144 N Central Ave Phoenix, AZ 85004
15      Blimpie Subs & Salads 4267 W Thunderbird Road ...
16      Cornish Pasty Company 960 W University Dr Temp...
17      Chuck E Cheese 6773 W Bell Road Glendale, AZ 8...
18        Kampai 2945 E Bell Rd Ste 102 Phoenix, AZ 85032
20      Paradise Bakery & Cafe 2370 W Happy Valley Rd ...
22      Draft House On The Reef 393 W Warner Rd Chandl...
26      Jody Maroni's Sausage Kingdom Sky Harbor Airpo...
28      Gallo Blanco Cafe 401 W Clarendon Ave Phoenix,...
33      Z'Tejas Southwestern Grill 20 W 6th St Tempe, ...
35      Zaytoon Kabob Bistro 4811 N 16th St Ste 104 Ph...
40      Great 

In [35]:
# Column with google search terms
restaurants_notmerged['google_term'] = restaurants_notmerged['name']+' '+restaurants_notmerged['full_address'].str.replace('\n',' ')
restaurants_notmerged['google_id'] = ''

In [532]:
# Google API
my_api_key = cred['API_Key']
my_cse_id = cred['CSE_id']

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res['items']

for i,restaurant in restaurants_notmerged.loc[3521:].iterrows():
    results = google_search(restaurant['google_term'], my_api_key, my_cse_id, num=1)
    if results[0]['link'].split('/')[3] == 'biz':
        restaurants_notmerged.loc[i,'google_id'] = results[0]['link'].split('/')[-1]
    else:
        raise ValueError('error at index: %d' % i)
    
    if i % 100 == 0:
        print(i)
    time.sleep(1.01) # rate limit is 100 requests per 100 seconds
    

In [529]:
restaurants_notmerged.loc[3519,'google_term']

'The Good Egg 14046 N Scottsdale Rd Scottsdale, AZ 85254'

In [530]:
restaurants_notmerged.loc[3519,'google_id'] = 'the-good-egg-hayden-peak-scottsdale'

In [474]:
#restaurants_notmerged.loc[2998,'full_address'] = '14870 N Northsight Blvd\nScottsdale, AZ 85260'

In [533]:
restaurants_notmerged.loc[3519:]

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,found,google_term,google_id
3519,nP9zQBRNSxlnU8BsbqYI5A,"[Breakfast & Brunch, Restaurants]",Scottsdale,"14046 N Scottsdale Rd\nScottsdale, AZ 85254",33.614031,-111.926969,The Good Egg,[],True,27,3.0,AZ,business,0,"The Good Egg 14046 N Scottsdale Rd Scottsdale,...",the-good-egg-hayden-peak-scottsdale
3521,N_zdv9ApM_RIk_80KsNX5g,"[Diners, Restaurants]",Peoria,"7541 W Bell Rd\nPeoria, AZ 85382",33.637664,-112.221341,5 & Diner,[],True,20,3.0,AZ,business,0,"5 & Diner 7541 W Bell Rd Peoria, AZ 85382",5-and-diner-peoria
3522,ImVx220_Pcp2iB5uworODw,"[Italian, Pizza, Restaurants]",Phoenix,"4905 E Ray Rd\nSte 104\nPhoenix, AZ 85044",33.319379,-111.977875,Vincent's,[],True,60,3.5,AZ,business,0,"Vincent's 4905 E Ray Rd Ste 104 Phoenix, AZ 85044",vincents-phoenix
3523,pbVL3t783OyO701yK5W08g,"[Delis, Italian, Restaurants]",Tempe,"1101 W University Dr\nTempe, AZ 85281",33.421730,-111.954320,Mama's Italian Deli,[],True,6,4.5,AZ,business,0,Mama's Italian Deli 1101 W University Dr Tempe...,mamas-italian-deli-tempe
3524,IkukcWBvBl10_5VPbx-Liw,"[Thai, Restaurants]",Chandler,"2820 S Alma School Rd\nChandler, AZ 85286",33.264433,-111.860100,Thai Dressing Fine Thai Cuisine,[],True,28,4.0,AZ,business,0,Thai Dressing Fine Thai Cuisine 2820 S Alma Sc...,thai-dressing-fine-thai-cuisine-chandler
3527,45LugFfNZ4fjbhypsmrLgQ,"[Breakfast & Brunch, Restaurants]",Phoenix,"Phoenix, AZ 85003",33.449188,-112.077200,The Good Egg,[],True,8,2.5,AZ,business,0,"The Good Egg Phoenix, AZ 85003",the-good-egg-phoenix
3529,JSl9JtunfJVfu9AwtiN1ww,"[Breakfast & Brunch, Restaurants]",Chandler,"2925 South Alma School Road\nChandler, AZ 85248",33.263460,-111.857496,Einstein Brothers Bagels,[],True,3,2.5,AZ,business,0,Einstein Brothers Bagels 2925 South Alma Schoo...,einstein-bros-bagels-chandler-3
3537,FeI75xIG8PF_XZ6P80gLBQ,"[Pubs, Irish, Nightlife, Bars, Restaurants]",Phoenix,"705 N 1st St\nPhoenix, AZ 85004",33.455863,-112.072167,The Turf,[],True,122,3.5,AZ,business,0,"The Turf 705 N 1st St Phoenix, AZ 85004",chambers-on-first-phoenix
3544,1_siFXaQbxck3tiKRwnq7Q,"[Breakfast & Brunch, Caterers, Sandwiches, Eve...",Tempe,"315 W Elliot Rd\nSte 107\nTempe, AZ 85284",33.348763,-111.944351,AZ Bread Company,[],True,81,4.0,AZ,business,0,AZ Bread Company 315 W Elliot Rd Ste 107 Tempe...,az-bread-tempe
3552,VD8a_B589J_NlCirS8QA0Q,"[Greek, Mediterranean, Restaurants]",Carefree,"37555 Hum Road\n# 109\nCarefree, AZ 85377",33.825686,-111.923149,Saba's Mediterranean Cuisine,[],True,20,4.0,AZ,business,0,Saba's Mediterranean Cuisine 37555 Hum Road # ...,sabas-mediterranean-cuisine-carefree


In [405]:
results[0]['link']

'https://www.yelp.com/biz/americas-taco-shop-phoenix-2'

In [527]:
i

3519

In [534]:
restaurants_notmerged.to_pickle('./yelp_dataset_processed/yelp_df_notmerged_id.pkl')